In [1]:
import warnings  

import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from torch.nn.functional import softmax

from efficientnet_pytorch import EfficientNet

from typing import Callable, List, Tuple, Dict
from pathlib import Path

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from collections import defaultdict, OrderedDict
from tqdm.notebook import tqdm
from torchsummary import summary

from fastai2.layers import LabelSmoothingCrossEntropy
from transformers import get_cosine_schedule_with_warmup
from transformers import AdamW

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [2]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
EXPERIMENT_NAME = "02_efficientnet-b6"

class ConfigExperiment:
    logdir = f"./logs/{EXPERIMENT_NAME}"
    save_dirname = EXPERIMENT_NAME
    submission_file = f"{EXPERIMENT_NAME}.csv"
    seed = 42
    batch_size = 2
    model_name = 'efficientnet-b6'
    size = 512
    num_workers = 20
    root_images = "../../../data/raw/plant-pathology-2020-fgvc7/images/"
    root = "../../../data/raw/plant-pathology-2020-fgvc7/"
    num_classes = 4
    patience = 10
    early_stopping_delta = 1e-4
    num_epochs = 300
    lr = 0.003
    class_names = ["healthy", "multiple_diseases", "rust", "scab"]
    is_fp16_used = False
    
    
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    
config = ConfigExperiment()
set_seed(config.seed)
config.size = EfficientNet.get_image_size(config.model_name)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    # Create target Directory
    os.mkdir(config.save_dirname)
    print("Directory " , config.save_dirname ,  " Created ") 
except FileExistsError:
    print("Directory " , config.save_dirname ,  " already exists")

Directory  02_efficientnet-b6  already exists


In [4]:
class PlantDataset(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.iloc[idx][self.class_names].values.astype(np.int8)
        label = torch.argmax(torch.from_numpy(labels))
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, label

In [5]:
def pre_transforms(image_size=224):
    # Convert the image to a square of size image_size x image_size
    # (keeping aspect ratio)
    result = [
        A.LongestMaxSize(max_size=image_size),
        A.PadIfNeeded(image_size, image_size, border_mode=0)
    ]
    
    return result

def hard_transforms():
    result = [
        # Random shifts, stretches and turns with a 50% probability
#         A.RandomResizedCrop(height=config.size, width=config.size, p=1.0),
        A.OneOf([
            A.Rotate(limit=90, p=1),
            A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1),
            A.OpticalDistortion(p=1),
            A.ShiftScaleRotate(rotate_limit=90, shift_limit=0.2, scale_limit=0.1, p=1)
        ], p=0.8),
        
        # Pixels
        A.OneOf([
            A.IAAEmboss(p=1.0),
            A.IAASharpen(p=1.0),
            A.Blur(p=1.0),
        ], p=0.5),

        # Affine
        A.OneOf([
            A.RandomContrast(limit=0.2, p=1),
            A.ElasticTransform(p=1),
            A.IAAPiecewiseAffine(p=1)
        ], p=0.5),
    ]
    
    return result

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    return [A.Normalize(p=1.0), ToTensorV2(p=1.0),]

def compose(transforms_to_compose):
    # combine all augmentations into one single pipeline
    result = A.Compose([item for sublist in transforms_to_compose for item in sublist])
    return result

In [6]:
train_df = pd.read_csv(config.root + 'train.csv')
train, valid = train_test_split(train_df, test_size=0.33, random_state=config.seed, shuffle=True, stratify=train_df[config.class_names])

train_transforms = compose([
    pre_transforms(config.size),
    hard_transforms(), 
    post_transforms()
])
valid_transforms = compose([
    pre_transforms(config.size), 
    post_transforms()
])

show_transforms = compose([
    pre_transforms(config.size),
    hard_transforms()
])

train_dataset = PlantDataset(train, config, train_transforms)
valid_dataset = PlantDataset(valid, config, valid_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [7]:
def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet") -> EfficientNet:
    model = EfficientNet.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model._fc.in_features
    model._fc = nn.Sequential(nn.Linear(num_ftrs, num_classes, bias = True))
    return model

model = get_model(config.model_name, config.num_classes)

Loaded pretrained weights for efficientnet-b6


In [8]:
criterion = LabelSmoothingCrossEntropy()
optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-3)

num_train_steps = int(len(train_dataset) / config.batch_size * config.num_epochs)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataset)/config.batch_size*5, num_training_steps=num_train_steps)

In [9]:
class Trainer:
    def __init__(self, model, train_dataloader: DataLoader, valid_dataloader: DataLoader, criterion, optimizer, scheduler, device, config: ConfigExperiment):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.config = config
        self.train_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.valid_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.counter = 0
        self.delta = config.early_stopping_delta
      
    def run(self):
        self.model.to(device)
        best_valid_loss = float('inf')
        best_valid_auc_mean = 0

        try:
            for i_epoch in tqdm(range(self.config.num_epochs), desc='Epochs', total=config.num_epochs, position=1, leave=True):
                start_time = time.time()

                train_loss, train_outputs, train_targets = self._train()
                valid_loss, valid_outputs, valid_targets = self._evaluate()
                    
                self.train_metrics["avg_loss"].append(train_loss)
                self.train_metrics["auc/_mean"].append(self.comp_metric(train_outputs, train_targets))
                self.train_metrics["auc/healthy"].append(self.healthy_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/rust"].append(self.rust_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/scab"].append(self.scab_roc_auc(train_outputs, train_targets))
                
                self.valid_metrics["avg_loss"].append(valid_loss)
                self.valid_metrics["auc/_mean"].append(self.comp_metric(valid_outputs, valid_targets))
                self.valid_metrics["auc/healthy"].append(self.healthy_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/rust"].append(self.rust_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/scab"].append(self.scab_roc_auc(valid_outputs, valid_targets))
                
                end_time = time.time()
                epoch_mins, epoch_secs = self._epoch_time(start_time, end_time)
                self.print_progress(i_epoch, epoch_mins, epoch_secs)
                
                self.scheduler.step(self.valid_metrics["auc/_mean"][-1])
                
                if valid_loss < best_valid_loss:
                    self.counter = 0
                    best_valid_loss = valid_loss
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_loss={best_valid_loss}.pth")
                else:
                    self.counter += 1
                    
                if self.valid_metrics["auc/_mean"][-1] > best_valid_auc_mean:
                    best_valid_auc_mean = self.valid_metrics["auc/_mean"][-1]
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_auc_mean={best_valid_auc_mean}.pth")
                    
                if self.counter > self.config.patience:
                    print("EarlyStopping")
                    break
        except KeyboardInterrupt: 
            pass
        
        return self.train_metrics, self.valid_metrics
        
    def _train(self):
        model.train()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        for i, (images, labels) in tqdm(enumerate(self.train_dataloader), desc='Train', total=len(self.train_dataloader), position=2, leave=True):
            loss_iten, outputs = self._train_process(images, labels)
            epoch_loss += loss_iten              

            if epoch_output is None:
                epoch_output = outputs.cpu().data
            else:
                epoch_output = torch.cat((epoch_output, outputs.cpu().data))

            if epoch_target is None:
                epoch_target = labels.cpu().data
            else:
                epoch_target = torch.cat((epoch_target, labels.cpu().data))
            
        return epoch_loss / len(self.train_dataloader), epoch_output, epoch_target
    
    def _train_process(self, images, labels):
        images = images.to(self.device)
        labels = labels.to(self.device)
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        return loss.item(), outputs
            
    def _evaluate(self):
        model.eval()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        with torch.no_grad():
            for i, (images, labels) in enumerate(self.valid_dataloader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_loss += loss.item()
                
                if epoch_output is None:
                    epoch_output = outputs.cpu().data
                else:
                    epoch_output = torch.cat((epoch_output, outputs.cpu().data))

                if epoch_target is None:
                    epoch_target = labels.cpu().data
                else:
                    epoch_target = torch.cat((epoch_target, labels.cpu().data))

        return epoch_loss / len(self.valid_dataloader), epoch_output, epoch_target
    
#     def _evaluate_tta(self):
#         n_samples = len(dataset)
#         y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)

#         model.eval()
#         model.to(device)
#         with torch.no_grad():
#             for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
#                 for i, (images, idx) in enumerate(dataloader):
#                     images = images.to(device)
#                     outputs = model(images)
#                     y_pred = F.softmax(outputs, dim=1)

#                     tta_index = i_epoch - 1
#                     start_index = (i % len(dataloader)) * config.batch_size
#                     end_index = min(start_index + config.batch_size, n_samples)
#                     batch_y_probas = y_pred.detach().cpu().numpy()                
#                     y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

#         return y_probas_tta

#     def show_history(self):
#         fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
#         clear_output(True)
#         ax[0].plot(history, label='train loss')
#         ax[0].set_xlabel('Batch')
#         ax[0].set_title('Train loss')
#         if train_history is not None:
#             ax[1].plot(train_history, label='general train history')
#             ax[1].set_xlabel('Epoch')
#         if valid_history is not None:
#             ax[1].plot(valid_history, label='general valid history')
#         plt.legend()
#         plt.show()

    
    def _epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def print_progress(self, i_epoch, epoch_mins, epoch_secs):
        i_epoch = i_epoch + 1
        print(f"Epoch: {i_epoch:02} | Time: {epoch_mins}m {epoch_secs}s")
        print("Training Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format(
                self.train_metrics['avg_loss'][-1], 
                self.train_metrics['auc/_mean'][-1],
                self.train_metrics['auc/healthy'][-1],
                self.train_metrics['auc/multiple_diseases'][-1],
                self.train_metrics['auc/rust'][-1],
                self.train_metrics['auc/scab'][-1],
            ))
        print("Evaluating Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format( 
                self.valid_metrics['avg_loss'][-1], 
                self.valid_metrics['auc/_mean'][-1],
                self.valid_metrics['auc/healthy'][-1],
                self.valid_metrics['auc/multiple_diseases'][-1],
                self.valid_metrics['auc/rust'][-1],
                self.valid_metrics['auc/scab'][-1],
            ))
        print()
        
    def comp_metric(self, preds, targs, labels=range(4)):
        preds = torch.sigmoid(preds)
        targs = torch.eye(4)[targs]
        return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])

    def healthy_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[0])

    def multiple_diseases_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[1])

    def rust_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[2])

    def scab_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[3])



In [ ]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 1m 53s
Training Results - Average Loss: 1.3882 | auc/_mean: 0.4891 | auc/healthy: 0.4666 | auc/multiple_diseases: 0.5480, auc/rust: 0.4957, auc/scab: 0.4460
Evaluating Results - Average Loss: 1.3894 | auc/_mean: 0.4869 | auc/healthy: 0.4384 | auc/multiple_diseases: 0.5740, auc/rust: 0.4730, auc/scab: 0.4623




Epoch: 02 | Time: 1m 53s
Training Results - Average Loss: 1.3862 | auc/_mean: 0.4822 | auc/healthy: 0.4963 | auc/multiple_diseases: 0.4881, auc/rust: 0.4941, auc/scab: 0.4503
Evaluating Results - Average Loss: 1.3893 | auc/_mean: 0.4949 | auc/healthy: 0.4485 | auc/multiple_diseases: 0.5624, auc/rust: 0.4870, auc/scab: 0.4818




Epoch: 03 | Time: 1m 53s
Training Results - Average Loss: 1.3865 | auc/_mean: 0.4603 | auc/healthy: 0.4562 | auc/multiple_diseases: 0.4460, auc/rust: 0.4908, auc/scab: 0.4481
Evaluating Results - Average Loss: 1.3873 | auc/_mean: 0.5018 | auc/healthy: 0.4594 | auc/multiple_diseases: 0.5605, auc/rust: 0.4944, auc/scab: 0.4929




Epoch: 04 | Time: 1m 53s
Training Results - Average Loss: 1.3802 | auc/_mean: 0.4954 | auc/healthy: 0.4808 | auc/multiple_diseases: 0.5482, auc/rust: 0.4919, auc/scab: 0.4607
Evaluating Results - Average Loss: 1.3868 | auc/_mean: 0.5083 | auc/healthy: 0.4599 | auc/multiple_diseases: 0.5642, auc/rust: 0.5042, auc/scab: 0.5050




Epoch: 05 | Time: 1m 53s
Training Results - Average Loss: 1.3789 | auc/_mean: 0.4940 | auc/healthy: 0.4828 | auc/multiple_diseases: 0.5505, auc/rust: 0.5098, auc/scab: 0.4329
Evaluating Results - Average Loss: 1.3844 | auc/_mean: 0.5123 | auc/healthy: 0.4645 | auc/multiple_diseases: 0.5575, auc/rust: 0.5150, auc/scab: 0.5124



...


Epoch: 161 | Time: 1m 53s
Training Results - Average Loss: 1.1444 | auc/_mean: 0.7686 | auc/healthy: 0.8133 | auc/multiple_diseases: 0.5765, auc/rust: 0.8307, auc/scab: 0.8541
Evaluating Results - Average Loss: 1.1581 | auc/_mean: 0.8293 | auc/healthy: 0.9191 | auc/multiple_diseases: 0.4939, auc/rust: 0.9532, auc/scab: 0.9509




Epoch: 162 | Time: 1m 52s
Training Results - Average Loss: 1.1460 | auc/_mean: 0.7614 | auc/healthy: 0.8112 | auc/multiple_diseases: 0.5544, auc/rust: 0.8352, auc/scab: 0.8450
Evaluating Results - Average Loss: 1.1605 | auc/_mean: 0.8299 | auc/healthy: 0.9184 | auc/multiple_diseases: 0.5011, auc/rust: 0.9504, auc/scab: 0.9497




Epoch: 163 | Time: 1m 53s
Training Results - Average Loss: 1.1445 | auc/_mean: 0.7542 | auc/healthy: 0.8154 | auc/multiple_diseases: 0.5078, auc/rust: 0.8310, auc/scab: 0.8625
Evaluating Results - Average Loss: 1.1601 | auc/_mean: 0.8249 | auc/healthy: 0.9194 | auc/multiple_diseases: 0.4806, auc/rust: 0.9507, auc/scab: 0.9489




Epoch: 164 | Time: 1m 53s
Training Results - Average Loss: 1.1485 | auc/_mean: 0.7622 | auc/healthy: 0.8386 | auc/multiple_diseases: 0.5523, auc/rust: 0.8140, auc/scab: 0.8439
Evaluating Results - Average Loss: 1.1588 | auc/_mean: 0.8303 | auc/healthy: 0.9182 | auc/multiple_diseases: 0.5018, auc/rust: 0.9514, auc/scab: 0.9498




Epoch: 165 | Time: 1m 53s
Training Results - Average Loss: 1.1531 | auc/_mean: 0.7505 | auc/healthy: 0.8252 | auc/multiple_diseases: 0.5308, auc/rust: 0.8124, auc/scab: 0.8337
Evaluating Results - Average Loss: 1.1567 | auc/_mean: 0.8313 | auc/healthy: 0.9189 | auc/multiple_diseases: 0.5058, auc/rust: 0.9506, auc/scab: 0.9501



In [10]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=165_auc_mean=0.8313264066195609.pth"))
for param in model.parameters():
    param.requires_grad = True

criterion = LabelSmoothingCrossEntropy()
optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-3)
num_train_steps = int(len(train_dataset) / config.batch_size * config.num_epochs)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataset)/config.batch_size*5, num_training_steps=num_train_steps)

Loaded pretrained weights for efficientnet-b6


In [11]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 5m 58s
Training Results - Average Loss: 1.1879 | auc/_mean: 0.7153 | auc/healthy: 0.7541 | auc/multiple_diseases: 0.5525, auc/rust: 0.7709, auc/scab: 0.7838
Evaluating Results - Average Loss: 1.2575 | auc/_mean: 0.8348 | auc/healthy: 0.9295 | auc/multiple_diseases: 0.5147, auc/rust: 0.9505, auc/scab: 0.9448




Epoch: 02 | Time: 5m 57s
Training Results - Average Loss: 1.1808 | auc/_mean: 0.7168 | auc/healthy: 0.7840 | auc/multiple_diseases: 0.5439, auc/rust: 0.7638, auc/scab: 0.7755
Evaluating Results - Average Loss: 1.2060 | auc/_mean: 0.8423 | auc/healthy: 0.9451 | auc/multiple_diseases: 0.5034, auc/rust: 0.9661, auc/scab: 0.9544



In [15]:

class PlantDatasetTest(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, idx

test_df = pd.read_csv(config.root + 'test.csv')
test_dataset = PlantDatasetTest(test_df, config, train_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [16]:
def predict(model, dataloader: DataLoader, device, dataset: Dataset, tta_count: int, config: ConfigExperiment):
    n_samples = len(dataset)
    y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)
    
    model.eval()
    model.to(device)
    with torch.no_grad():
        for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
            for i, (images, idx) in enumerate(dataloader):
                images = images.to(device)
                outputs = model(images)
                y_pred = F.softmax(outputs, dim=1)
                
                tta_index = i_epoch - 1
                start_index = (i % len(dataloader)) * config.batch_size
                end_index = min(start_index + config.batch_size, n_samples)
                batch_y_probas = y_pred.detach().cpu().numpy()                
                y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

    return y_probas_tta

In [17]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=39_auc_mean=0.9220583291148317.pth"))

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [18]:
result = predict(model, test_dataloader, device, test_dataset, tta_count=5, config=config)

In [19]:
y_probas = np.mean(result, axis=-1)

In [20]:
y_probas[:10]

array([[9.9273829e-04, 2.2985693e-02, 9.7583580e-01, 1.8573811e-04],
       [3.9310761e-02, 1.7064143e-02, 9.4097364e-01, 2.6514246e-03],
       [9.1703532e-06, 1.3889124e-03, 5.8242813e-06, 9.9859619e-01],
       [9.9602592e-01, 6.7340222e-04, 7.6660613e-04, 2.5340687e-03],
       [9.4989018e-06, 2.8524425e-02, 9.7144318e-01, 2.2904725e-05],
       [4.9693507e-01, 1.8131445e-01, 1.2418524e-01, 1.9756520e-01],
       [9.9773788e-01, 7.0325274e-04, 6.9397263e-04, 8.6491025e-04],
       [3.9168470e-05, 5.9642859e-02, 9.4551471e-04, 9.3937242e-01],
       [3.3926888e-05, 4.5833280e-03, 8.5793276e-05, 9.9529696e-01],
       [1.2231877e-04, 1.2349771e-02, 9.8747653e-01, 5.1319435e-05]],
      dtype=float32)

In [21]:
test_df = pd.read_csv(config.root + 'test.csv')
test_df["healthy"] = 0
test_df["multiple_diseases"] = 0
test_df["rust"] = 0
test_df["scab"] = 0
test_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = y_probas
test_df.to_csv(config.submission_file, index=False)
test_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.000993,0.022986,0.975836,0.000186
1,Test_1,0.039311,0.017064,0.940974,0.002651
2,Test_2,0.000009,0.001389,0.000006,0.998596
3,Test_3,0.996026,0.000673,0.000767,0.002534
4,Test_4,0.000009,0.028524,0.971443,0.000023


In [22]:
config.submission_file

'01_starter_code.csv'